In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

TIME_INTERVAL = 1/30
features = [
    'time',
    'LocationX',
    'LocationY',
    'LocationZ',
    'LaunchX',
    'LaunchY',
    'LaunchZ',
    'LaunchAngle',
    'LaunchDirection',
    'InitialV',
    'label'
]

In [149]:
dataset= pd.read_excel("../testData/Vision Trajectory3.xlsx", sheet_name = ['forward3_out','forward7_out','forward6_scaled','forward7_scaled','shot1_away_camera'])

combinedDF = pd.DataFrame()
for feature in features:
    combinedDF[feature]= None


for sheet in dataset:
    print(sheet)
    # get each sheet
    DF = dataset[sheet]
    DF= DF[DF['Ball']==1]

    # create temporary dataframe and populate it with features
    tempDF = pd.DataFrame()
    for feature in features:
        tempDF[feature]= None
    tempDF['LocationX'] = DF['y'] * 10
    tempDF['LocationY'] = DF['x'] * 10
    tempDF['LocationZ'] = DF['z'] * 10
    tempDF=findStart(tempDF) # overwrite
    tempDF.reset_index(drop=True)
    tempDF['time'] = np.arange(0.0,len(tempDF)*TIME_INTERVAL,TIME_INTERVAL)
    combinedDF=pd.concat([combinedDF,tempDF],ignore_index=True)



forward3_out
forward7_out
forward6_scaled
forward7_scaled
shot1_away_camera


In [150]:
combinedDF

,time,LocationX,LocationY,LocationZ,LaunchX,LaunchY,LaunchZ,LaunchAngle,LaunchDirection,InitialV,label
0,0.000000,2383.24,297.32,3318.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.033333,3167.01,292.04,3856.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.066667,3929.38,292.45,4345.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.100000,4505.53,311.43,4718.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.133333,4997.76,312.07,5030.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
270,1.366667,11821.38,-1410.99,4367.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,1.400000,11890.80,-1402.09,4244.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272,1.433333,11966.62,-1402.03,4110.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273,1.466667,12037.49,-1401.93,3983.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
#useless
def fillTime(dataframe,interval=TIME_INTERVAL):
    print(len(dataframe))
    t = 0.0
    for i in range(len(dataframe)):
        dataframe.iloc[i]['time'] = t
        t+=interval
    return dataframe


In [23]:
dataset

{'forward3_out':     Time  Ball       x         y        z
 0    0.0     1  27.454   157.111  278.883
 1    NaN     1  29.732   238.324  331.821
 2    NaN     1  29.204   316.701  385.695
 3    NaN     1  29.245   392.938  434.559
 4    NaN     1  31.143   450.553  471.839
 ..   ...   ...     ...       ...      ...
 56   NaN     1  80.571  1398.846  308.388
 57   NaN     1  80.095  1397.023  289.030
 58   NaN     1  81.437  1402.555  269.514
 59   NaN     1  80.779  1407.877  248.740
 60   NaN     1  82.035  1421.824  230.357
 
 [61 rows x 5 columns],
 'forward7_out':     Frame  Ball       x         y        z
 0     0.0     1   4.827   181.438  263.932
 1     NaN     1  18.452   257.894  303.579
 2     NaN     1  18.490   353.925  361.766
 3     NaN     1  19.219   436.240  410.769
 4     NaN     1  20.763   506.795  451.810
 5     NaN     1  21.709   567.025  486.064
 6     NaN     1  22.920   617.281  513.163
 7     NaN     1  24.829   669.909  541.393
 8     NaN     1  25.904   713

In [62]:
def plotSpeeds(dataframe):
    '''
    Calculates and plots instantaneous speed of shuttlecock between every frame
    '''
    speeds = []
    # we minus one since we are unable to access the coordinates at t+1 for the very last row, DNE! 
    for i in range(len(dataframe)-1):
        row = dataframe.iloc[i]
        nextRow = dataframe.iloc[i+1]
        forwardSpeed = (math.sqrt((nextRow['LocationX'] - row['LocationX'])**2 + 
                                  (nextRow['LocationY'] - row['LocationY'])**2 +
                                  (nextRow['LocationZ'] - row['LocationZ'])**2)) /(TIME_INTERVAL)
        
        speeds.append(forwardSpeed)
    fig = plt.Figure()
    plt.plot(speeds)
    plt.xlabel('Frame')
    plt.ylabel('Speed')
    plt.title('Instantaneous speed of shuttlecock at each frame')
    plt.grid(True)
    plt.show()

In [80]:
def findStart(dataframe):
    '''
    Find the true start point of a trajectory by looking at acceleration values between each of the points.
    The true start is the first frame when the shuttlecock has just left the racket. Assume that happens when shuttlecock reaches its max instantaneous velocity.
    Then the true start point should be the i+1 point! Since the speed btwn i+2 and i+1 is less than btwn i+1 and i, it makes sense that
    shuttlecock left at i+1!
    '''
    speeds = []
    # we minus one since we are unable to access the coordinates at t+1 for the very last row, DNE! 
    for i in range(len(dataframe)-1):
        row = dataframe.iloc[i]
        nextRow = dataframe.iloc[i+1]
        forwardSpeed = (math.sqrt((nextRow['LocationX'] - row['LocationX'])**2 + 
                                  (nextRow['LocationY'] - row['LocationY'])**2 +
                                  (nextRow['LocationZ'] - row['LocationZ'])**2)) /(TIME_INTERVAL)
        speeds.append((forwardSpeed,i,i+1))
    speeds.sort(key= lambda x:x[0],reverse=True)
    trueStartIndex = speeds[0][2]
    return dataframe[trueStartIndex:]
    
        